In [1]:
%load_ext autoreload
%autoreload 2

import os
if os.getcwd() == '/home/jovyan/work': # jhub
    os.chdir("24FS_I4DS27/main/") 
    os.system("make reqs")
else: # local
    os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

In [21]:
# read in the results
results = defaultdict(list)
for folder in os.listdir("robustified_models"):

    folder_split = folder.split("-")
    model = f"{folder_split[0]}-{folder_split[1]}-{folder_split[2]}"
    i = folder_split[3].split("_")[1]

    for subfolder in os.listdir(f"robustified_models/{folder}"):
        if subfolder == "01_UAPs_pre_robustification":
            if os.path.exists(f"robustified_models/{folder}/{subfolder}/UAPs_tensor.pt"):
                csv_file = f"robustified_models/{folder}/{subfolder}/metrics.csv"
                metrics_df = pd.read_csv(csv_file)
                uap_time_elapsed = metrics_df["uap_time_elapsed"].dropna()
                results[(model, i, subfolder, "uap_time_elapsed", "mean")] = uap_time_elapsed.mean()
                results[(model, i, subfolder, "uap_time_elapsed", "std")] = uap_time_elapsed.std()
            else:
                results[(model, i, subfolder, "uap_time_elapsed", "mean")] = np.inf
                results[(model, i, subfolder, "uap_time_elapsed", "std")] = np.inf

        if subfolder.startswith(("02", "03", "05", "06")):
            csv_file = f"robustified_models/{folder}/{subfolder}/metrics.csv"
            metrics_df = pd.read_csv(csv_file).iloc[:, 2:]
            for col in metrics_df.columns:
                results[(model, i, subfolder, col, None)] = metrics_df.loc[:, col].values[0]

        if subfolder.startswith(("03", "06")):
            no_perturbations_foldername = (
                "02_eval_unrobustified_test" if subfolder.startswith("03") else "05_eval_robustified_test"
            )
            predictions_UAP_df = pd.read_csv(f"robustified_models/{folder}/{subfolder}/test_predictions.csv")
            predictions_df = pd.read_csv(
                f"robustified_models/{folder}/{no_perturbations_foldername}/test_predictions.csv"
            )

            results[(model, i, subfolder, "fooling_rate", None)] = (predictions_UAP_df.predictions.round() != predictions_df.predictions.round()
            ).mean()
            # results[(model, i, subfolder, "predictions", None)] = list(predictions_df.iloc[:, 1].values)
            # results[(model, i, subfolder, "targets", None)] = list(predictions_df.iloc[:, 2].values.astype(int))

        if subfolder.startswith("04"):
            csv_file = f"robustified_models/{folder}/{subfolder}/metrics.csv"
            metrics_df = pd.read_csv(csv_file).iloc[:, 9:]
            for col in metrics_df.columns:
                results[(model, i, subfolder, col, None)] = metrics_df.loc[:, col].dropna().values

df_results = pd.DataFrame.from_dict(
    results,
    orient="index",
    columns=["value"],
)
df_results.index = pd.MultiIndex.from_tuples(
    df_results.index,
    names=["model", "robustification", "stage", "metric", "agg"],
)
df_results = df_results.sort_values(
    by=["model", "robustification", "stage", "metric", "agg"],
)
df_results = df_results.reset_index()

In [23]:
df_results[df_results["metric"] == "fooling_rate"].sort_values(by=["robustification", "stage"], ascending=True)

,model,robustification,stage,metric,agg,value
9,resnet50-covidx_data-n_100,0,03_eval_unrobustified_test_uap_0,fooling_rate,NaN,0.356755
17,resnet50-covidx_data-n_100,0,03_eval_unrobustified_test_uap_1,fooling_rate,NaN,0.349092
25,resnet50-covidx_data-n_100,0,03_eval_unrobustified_test_uap_2,fooling_rate,NaN,0.429262
33,resnet50-covidx_data-n_100,0,03_eval_unrobustified_test_uap_3,fooling_rate,NaN,0.436572
41,resnet50-covidx_data-n_100,0,03_eval_unrobustified_test_uap_4,fooling_rate,NaN,0.343787
63,resnet50-covidx_data-n_100,0,06_eval_robustified_test_uap_0,fooling_rate,NaN,0.171658
71,resnet50-covidx_data-n_100,0,06_eval_robustified_test_uap_1,fooling_rate,NaN,0.168121
79,resnet50-covidx_data-n_100,0,06_eval_robustified_test_uap_2,fooling_rate,NaN,0.218345
87,resnet50-covidx_data-n_100,0,06_eval_robustified_test_uap_3,fooling_rate,NaN,0.219524
95,resnet50-covidx_data-n_100,0,06_eval_robustified_test_uap_4,fooling_rate,NaN,0.165409
